# Natural Language Processing
[BPE Paper Link](https://arxiv.org/pdf/1508.07909.pdf) 

In [ ]:
'''
자연어 처리에서의 BPE는 서브워드 분리(subword segmentation) 알고리즘임.
꽤나 많이 쓰인다고 함 
기존에 있던 단어를 분리한다는 의미.
BPE는 글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용함.
주어진 단어들을 모든 글자(chracters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만든 후에
가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합하는 과정 반복함.

기존의 접근
----------------------------------------------------------
주어진 dict
low : 5, lower : 2, newest : 6, widest : 3

생성한 vocab
low, lower, newest, widest 임.
----------------------------------------------------------


BPE 알고리즘을 사용한 경우
----------------------------------------------------------
주어진 dict
l o w : 5,  l o w e r : 2,  n e w e s t : 6,  w i d e s t : 3

생성한 vocab
l, o, w, e, r, n, w, s, t, i, d
----------------------------------------------------------
즉, 그냥 나온 알파벳만 가져온 것이 초기 vocabulary가 됨
그 이후로 미리 설정한 반복수만큼
가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 반복함.
예시로, 위의 경우에서 1회차 수행은 다음과 같음
-----------------------------------------------------------
주어진 dict
l o w : 5, l o w e r : 2, n e w es t : 6, w i d es t : 3

생성한 vocab
l, o, w, e, r, n, w, s, t, i, d, es
-----------------------------------------------------------
이 때의 반복 횟수에 따라, er이나 est와 같은 유니그램이 만들어지면서
예를 들어 newer 나 lowest와 같은 등록되어있지 않은 vocab이 출현한 경우에도
new+er 나 low+est와 같은 식으로 인코딩 할 수 있게 되고, OOV를 피해갈 수 있게됨.


보다 자세한 설명은 reference 참고 
ref : https://wikidocs.net/22592
'''

from typing import List, Dict, Set
from itertools import chain
import re
from collections import defaultdict, Counter


def build_bpe(
        corpus: List[str],
        max_vocab_size: int
) -> List[int]:
    """ BPE Vocabulary Builder
    Implement vocabulary builder for byte pair encoding.
    Please sort your idx2word by subword length in descending manner.

    Hint: Counter in collection library would be helpful

    Note: If you convert sentences list to word frequence dictionary,
          building speed is enhanced significantly because duplicated words are
          preprocessed together

    Arguments:
    corpus -- List of words to build vocab
    max_vocab_size -- The maximum size of vocab

    Return:
    idx2word -- Subword list
    """
    # Special tokens
    PAD = BytePairEncoding.PAD_token  # Index of <PAD> must be 0
    UNK = BytePairEncoding.UNK_token  # Index of <UNK> must be 1
    CLS = BytePairEncoding.CLS_token  # Index of <CLS> must be 2
    SEP = BytePairEncoding.SEP_token  # Index of <SEP> must be 3
    MSK = BytePairEncoding.MSK_token  # Index of <MSK> must be 4
    SPECIAL = [PAD, UNK, CLS, SEP, MSK]

    WORD_END = BytePairEncoding.WORD_END  # Use this token as the end of a word


    def get_stats(vocab):
        stats = defaultdict(int)
        for word, freq in vocab.items(): #현재 step의 vocab 가져오기
            symbols = word.split() #띄어쓰기 된 녀석들 symbols에 넣기
            for i in range(len(symbols)-1):
                stats[symbols[i],symbols[i+1]] += freq #빈도수 측정
        return stats


    def replace_pair(most_freq, vocab_dict):
        replaced_vocab = {}
        bigram = re.escape(' '.join(most_freq))
        
        
        patt = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') #bigram이 b c일 때 a b c d , b c 같은 형태만 매칭하고  ab c d , b cd 같은 형태는 매칭하지 않을려고 사용
                                                          #ref:https://blog.hexabrain.net/205, 김규진_T1011 캠퍼님
        for word in vocab_dict :
            w_out = patt.sub(''.join(most_freq), word)
            replaced_vocab[w_out] = vocab_dict[word] #떨어져 있던 녀석들 붙여서 넘겨주기
        return replaced_vocab

    #counter로 vocab별 출현횟수 가져오기
    vocabs = Counter(corpus)
    
    vocab_list = []
    for v, v_count in vocabs.items() :
        #알파벳 사이에 띄어쓰기 넣고(맨 끝에도), 해당 vocab가 끝났다는 의미의 _ 붙여놓기 
        vocab_list.append((str(' '.join(v[:-1])) + ' ' + str(v[-1] + ' _'), v_count))
    #print(vocab_list)
    vocab_dict = dict(vocab_list)

    idx2word = []

    #각 vocab별로 나온 token_list
    for token_list in vocab_dict.keys() :
        #띄어쓰기로 나눠놨던 각 토큰
        for token in token_list.split() :
            #만약 해당 토큰이 idx2word에 없다면 추가
            if token not in idx2word :
                idx2word.append(token)
    
    for i in range(max_vocab_size):
        #토큰쌍 출현 정보 가져오기 
        stats = get_stats(vocab_dict)
        #토큰쌍 출현 정보가 없대 ~ split 될게 없어 이미 다 끝났어 (max_vocab_size가 크게 설정되었음)
        if len(stats) == 0: 
            break #그럼끝
        #print(stats) 
        most_freq = max(stats, key=stats.get) #가장 빈번한 녀석
        #print(most_freq)
        #print("replace 이전 vocab_dict :",vocab_dict)
        vocab_dict = replace_pair(most_freq, vocab_dict)
        #print("replace 이후 vocab_dict :",vocab_dict)
        #가장 빈번한 토큰쌍 붙이고 idx2word에 저장
        idx2word.append(''.join(most_freq))
        #완료조건
        if len(idx2word) == max_vocab_size - 5 :
            break

    #special 토큰들 넣고 긴 순서부터 길이에 맞춰 정렬 후 return
    idx2word = SPECIAL + sorted(idx2word, key=len, reverse=True)
    return idx2word

In [ ]:
#############################################
# Helper functions below. DO NOT MODIFY!    #
#############################################

class BytePairEncoding(object):
    """ Byte Pair Encoding class
    We aren't gonna use this class for encoding. Because it is too slow......
    We will use sentence piece Google have made.
    Thus, this class is just for special token index reference.
    """
    PAD_token = '<pad>'
    PAD_token_idx = 0
    UNK_token = '<unk>'
    UNK_token_idx = 1
    CLS_token = '<cls>'
    CLS_token_idx = 2
    SEP_token = '<sep>'
    SEP_token_idx = 3
    MSK_token = '<msk>'
    MSK_token_idx = 4

    WORD_END = '_'

    def __init__(self, corpus: List[List[str]], max_vocab_size: int) -> None:
        self.idx2word = build_bpe(corpus, max_vocab_size)

    def encode(self, sentence: List[str]) -> List[int]:
        return encode(sentence, self.idx2word)

    def decoder(self, tokens: List[int]) -> List[str]:
        return decode(tokens, self.idx2word)

======Building BPE Vocab Test Case======
The first test passed!
The second test passed!
['<pad>', '<unk>', '<cls>', '<sep>', '<msk>', 'abcde', 'abcd', 'abc', 'ab', 'a', 'b', 'c', 'd', 'e', '_']
The third test passed!
{'r', 'newest_', '<sep>', '_', 'est_', 'low', 'o', 'lo', 'e', 'i', 'd', 'l', 'est', 'new', '<unk>', 'n', 'ne', 't', 'es', '<msk>', '<cls>', 'w', '<pad>', 's'}
The forth test passed!
{'ab', 'aa', '<msk>', 'aaaaaaaa', 'aaaa', 'b', '<unk>', '<sep>', '<cls>', 'a', '_', '<pad>', 'abab'}
The fifth test passed!
['<pad>', '<unk>', '<cls>', '<sep>', '<msk>', 'abc_', 'bcd_', 'abc', 'bcd', 'bc', 'a', 'b', 'c', '_', 'd']
All 5 tests passed!
